## Example spatially explicit neutral model simulation

This notebook is intended as an example of how the data contained in results was generated. As many thousands of simulations were performed to generate the result set using high throughput computing systems at Imperial College London, the full code for performing these simulations cannot be run locally. 

In [1]:
import os
from pycoalescence import Simulation, CoalescenceTree, Map, DispersalSimulation
import pandas as pd

In [2]:
# Set the map file variable - in the main simulation pipeline, this would be changed to iterate
# over all required files
map_file = os.path.join("data", "map_50_0.2_0.tif")
output_dir = os.path.join("results", "example")
# This variable would change depending on the type of the map file
sim_type = "real"

In [3]:
# reference variable for file naming
job_number = 1 
# list containing output data
output = []
# Set the random seed
seed = 1
for sigma in [2, 4, 8, 16, 32]:
	sim = Simulation(logging_level=50)
	sim.set_simulation_parameters(seed=seed, job_type=job_number,
								  output_directory=output_dir,
								  min_speciation_rate=0.0001, sigma=sigma, deme=1, sample_size=1.0,
								  max_time=70 * 60 * 60, dispersal_method="normal",
								  landscape_type="tiled_fine")
	sim.set_speciation_rates([0.0001, 0.001, 0.01, 0.1])
	sim.set_map_files(sample_file="null", fine_file=map_file)
	# Remove the output, if it already exists
	if os.path.exists(sim.output_database):
		os.remove(sim.output_database)
	# Run the simulation
	sim.run()
	# Extract the species richness for each speciation rate and append it to the output 
	t = CoalescenceTree(sim)
	for ref in t.get_community_references():
		richness = t.get_species_richness(ref)
		output.append({"file" : map_file, "seed" : seed, "job_type" : job_number, "sigma" : sigma,
					   "speciation_rate" : t.get_community_parameters(ref)["speciation_rate"],
					   "richness" : richness, "type" : sim_type
					   })
	seed += 1
# Write the output to a csv (using a pandas data frame)
output_df = pd.DataFrame(output)
output_df.to_csv(os.path.join(output_dir, "variable_sizes_{}.csv".format(job_number)))

## Example dispersal simulation

Example of the dispersal simulation (the calculation of the $\mu_n$ metric required for $\sigma_e$
(and therefore $c_e$).

In [6]:
output = []
seed = 1
output_file = os.path.join(output_dir, "dispersal_{}_{}.db".format(sim_type, seed))
if os.path.exists(output_file):
	os.remove(output_file)
d = DispersalSimulation(logging_level=50)
d.set_simulation_parameters(number_repeats=100, output_database=output_file,
							number_steps=10000, seed=seed, sigma=2, 
							landscape_type="tiled_fine")

d.set_map(map_file)
# Get the dimensions of the map
m = Map(map_file)
x, y = m.get_x_y()
a_max = x * y
# Remove the output database, if it exists
for sigma in [2, 4, 8, 16, 32]:
	d.update_parameters(sigma=sigma)
	d.run_mean_distance_travelled()
for ref in d.get_database_references():
	params = d.get_database_parameters(ref)
	distance = d.get_mean_distance_travelled()
	sigma = params["sigma"]
	output.append({"type" : sim_type, "file" : map_file, "sigma" : sigma, 
				   "mean_distance_10000" : distance, "a_max" : a_max})
output_df = pd.DataFrame(output)
output_df.to_csv(os.path.join(output_dir, "variable_sizes_dispersal_{}.csv".format(job_number)))